In [1]:
import os
import pandas as pd

base_dir = "../Data"
base_csv_path = os.path.join(base_dir, 'base_cleaned.csv')
assert os.path.exists(base_csv_path), f"base {base_csv_path} does not exist"

output_csv_path = os.path.join(base_dir, 'm0_imputed.csv')
df = pd.read_csv(base_csv_path)

In [2]:
# Find the categorical columns using nunique
categorical_columns = df.nunique()[df.nunique() < 10].index.tolist()
categorical_column_indexes = [df.columns.get_loc(c) for c in categorical_columns if c in df]

# Print the number of categorical columns
print(f"Number of categorical columns: {len(categorical_columns)}")
print(f"Categorical columns: {categorical_columns}")

Number of categorical columns: 25
Categorical columns: ['thoracic_dgn', 'num_prev_tx', 'tah', 'vas', 'onvent', 'icu', 'inotropic', 'gender', 'abo', 'education', 'ecmo_tcr', 'iabp_tcr', 'inotropes_tcr', 'func_stat_tcr', 'diab', 'dial_ty_tcr', 'cereb_vasc', 'malig_tcr', 'cig_use', 'prior_card_surg_tcr', 'histry_cig_old', 'init_stat', 'ethcat', 'ventilator_tcr', 'work_income_tcr']


In [3]:
print(
    f"Total number of missing values in categorical columns before replacing missing values with 'missing': {df[categorical_columns].isnull().sum().sum()}")

# For all categorical columns, replace missing values with the string "missing"
for col in categorical_columns:
    df[col] = df[col].fillna("missing")

print(
    f"Total number of missing values in categorical columns after replacing missing values with 'missing': {df[categorical_columns].isnull().sum().sum()}")


Total number of missing values in categorical columns before replacing missing values with 'missing': 451455
Total number of missing values in categorical columns after replacing missing values with 'missing': 0


In [4]:
from sklearn.impute import SimpleImputer

print(f"Number of columns before imputation: {len(df.columns)}")
imputer = SimpleImputer(strategy='median')

print(f"Number of missing values before imputation: {df.isnull().sum().sum()}")
df_num = df.drop(columns=categorical_columns)  # Drop categorical columns
df_cat = df[categorical_columns]  # Keep categorical columns

df_num = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns,
                      index=df_num.index)  # Impute missing values in df_num
df = pd.concat([df_num, df_cat], axis=1)  # Merge to final dataframe

print(f"Number of missing values after imputation: {df.isnull().sum().sum()}")
print(f"Number of columns after imputation: {len(df.columns)}")

Number of columns before imputation: 34
Number of missing values before imputation: 78072
Number of missing values after imputation: 0
Number of columns after imputation: 34


In [5]:
# One hot encode the categorical columns
df = pd.get_dummies(df, columns=categorical_columns)
df.to_csv(output_csv_path, index=False)